# Import

In [46]:
import csv
import pandas as pd
from sqlalchemy import create_engine
import os
import sys
os.getcwd()
sys.path.append("C:/SQL_PW/")
import config 

# Read and Store CSV into Data Frame

In [2]:
#Read data
# Name of the CSV file
file1 = 'resources/us_food_stamps_by_state.csv'

# Read csv into pandas dataframe 
imported_data = pd.read_csv(file1)
imported_data.head()

,SUPPLEMENTAL NUTRITION ASSISTANCE PROGRAM: NUMBER OF PERSONS PARTICIPATING,"Resident population per state as of July 1, 2008","NUMBER OF PEOPLE, Jan 2008",Jan 2008 as % total pop,Dec 2008 (Preliminary),December 2008 as % total pop,Jan 2009 (initial),Jan 2009 as % total pop,% Change Dec 08 – Jan 09,% change Jan 08 – Jan 09
0,.Alabama,4661900,564325,12.105043,641449,13.759390,647628,13.891932,1.0,14.8
1,.Alaska,686293,56031,8.164297,57411,8.365377,61202,8.917765,6.6,9.2
2,.Arizona,6500180,602007,9.261390,742936,11.429468,756960,11.645216,1.9,25.7
3,.Arkansas,2855390,379031,13.274229,400571,14.028592,399243,13.982083,-0.3,5.3
4,.California,36756666,2160179,5.876972,2502279,6.807688,2545129,6.924265,1.7,17.8


# Clean Data

In [65]:

data_df = imported_data

# Using .rename(columns={}) in order to rename columns
renamed_df = data_df.rename(columns={"SUPPLEMENTAL NUTRITION ASSISTANCE PROGRAM: NUMBER OF PERSONS PARTICIPATING":"State", "Resident population per state as of July 1, 2008":"Residents_July_1_2008", "NUMBER OF PEOPLE, Jan 2008":"Fs_Pop", "Jan 2008 as % total pop":"PCTP_Jan_2008", "Dec 2008 (Preliminary)":"TP_Dec_2008", "December 2008 as % total pop":"PCTP_Dec_2008", "Jan 2009 (initial)":"TP_Jan_2009", "Jan 2009 as % total pop":"PCTP_Jan_2009", "% Change Dec 08 – Jan 09":"PCCH_Dec08_Jan09", "% change Jan 08 – Jan 09":"PCCH_Jan08_Jan09" })
renamed_df.head()


,SUPPLEMENTAL NUTRITION ASSISTANCE PROGRAM: NUMBER OF PERSONS PARTICIPATING,Residents_July_1_2008,Fs_Pop,PCTP_Jan_2008,TP_Dec_2008,PCTP_Dec_2008,TP_Jan_2009,PCTP_Jan_2009,PCCH_Dec08_Jan09,PCCH_Jan08_Jan09
0,.Alabama,4661900,564325,12.105043,641449,13.759390,647628,13.891932,1.0,14.8
1,.Alaska,686293,56031,8.164297,57411,8.365377,61202,8.917765,6.6,9.2
2,.Arizona,6500180,602007,9.261390,742936,11.429468,756960,11.645216,1.9,25.7
3,.Arkansas,2855390,379031,13.274229,400571,14.028592,399243,13.982083,-0.3,5.3
4,.California,36756666,2160179,5.876972,2502279,6.807688,2545129,6.924265,1.7,17.8


In [66]:
#To debug the unreplaced column name, show the names of the column
renamed_df.columns

Index(['SUPPLEMENTAL NUTRITION ASSISTANCE PROGRAM:  NUMBER OF PERSONS PARTICIPATING',
       'Residents_July_1_2008', 'Fs_Pop', 'PCTP_Jan_2008', 'TP_Dec_2008',
       'PCTP_Dec_2008', 'TP_Jan_2009', 'PCTP_Jan_2009', 'PCCH_Dec08_Jan09',
       'PCCH_Jan08_Jan09'],
      dtype='object')

In [67]:
# know the type of data
renamed_df.dtypes

SUPPLEMENTAL NUTRITION ASSISTANCE PROGRAM:  NUMBER OF PERSONS PARTICIPATING     object
Residents_July_1_2008                                                            int64
Fs_Pop                                                                           int64
PCTP_Jan_2008                                                                  float64
TP_Dec_2008                                                                      int64
PCTP_Dec_2008                                                                  float64
TP_Jan_2009                                                                      int64
PCTP_Jan_2009                                                                  float64
PCCH_Dec08_Jan09                                                               float64
PCCH_Jan08_Jan09                                                               float64
dtype: object

In [54]:
# replace that column name
renamed_df.rename(columns={'SUPPLEMENTAL NUTRITION ASSISTANCE PROGRAM:  NUMBER OF PERSONS PARTICIPATING': 'state'}, inplace=True)
renamed_df.head()

,state,Residents_July_1_2008,Fs_Pop,PCTP_Jan_2008,TP_Dec_2008,PCTP_Dec_2008,TP_Jan_2009,PCTP_Jan_2009,PCCH_Dec08_Jan09,PCCH_Jan08_Jan09
0,.Alabama,4661900,564325,12.105043,641449,13.759390,647628,13.891932,1.0,14.8
1,.Alaska,686293,56031,8.164297,57411,8.365377,61202,8.917765,6.6,9.2
2,.Arizona,6500180,602007,9.261390,742936,11.429468,756960,11.645216,1.9,25.7
3,.Arkansas,2855390,379031,13.274229,400571,14.028592,399243,13.982083,-0.3,5.3
4,.California,36756666,2160179,5.876972,2502279,6.807688,2545129,6.924265,1.7,17.8


In [55]:
# rename by creating a new column and removing anything that is not numeric or alphabetical as well as space 
renamed_df["State"] = renamed_df['state'].str.replace('[^\w\s]','')

In [56]:
renamed_df.head()

,state,Residents_July_1_2008,Fs_Pop,PCTP_Jan_2008,TP_Dec_2008,PCTP_Dec_2008,TP_Jan_2009,PCTP_Jan_2009,PCCH_Dec08_Jan09,PCCH_Jan08_Jan09,State
0,.Alabama,4661900,564325,12.105043,641449,13.759390,647628,13.891932,1.0,14.8,Alabama
1,.Alaska,686293,56031,8.164297,57411,8.365377,61202,8.917765,6.6,9.2,Alaska
2,.Arizona,6500180,602007,9.261390,742936,11.429468,756960,11.645216,1.9,25.7,Arizona
3,.Arkansas,2855390,379031,13.274229,400571,14.028592,399243,13.982083,-0.3,5.3,Arkansas
4,.California,36756666,2160179,5.876972,2502279,6.807688,2545129,6.924265,1.7,17.8,California


In [57]:
# Delete extraneous column
del renamed_df['state']
renamed_df.head()

,Residents_July_1_2008,Fs_Pop,PCTP_Jan_2008,TP_Dec_2008,PCTP_Dec_2008,TP_Jan_2009,PCTP_Jan_2009,PCCH_Dec08_Jan09,PCCH_Jan08_Jan09,State
0,4661900,564325,12.105043,641449,13.759390,647628,13.891932,1.0,14.8,Alabama
1,686293,56031,8.164297,57411,8.365377,61202,8.917765,6.6,9.2,Alaska
2,6500180,602007,9.261390,742936,11.429468,756960,11.645216,1.9,25.7,Arizona
3,2855390,379031,13.274229,400571,14.028592,399243,13.982083,-0.3,5.3,Arkansas
4,36756666,2160179,5.876972,2502279,6.807688,2545129,6.924265,1.7,17.8,California


In [58]:
#rearrange the column
cleaned_df = renamed_df[["State", "Residents_July_1_2008", "Fs_Pop", "PCTP_Jan_2008", "TP_Dec_2008", "PCTP_Dec_2008", "TP_Jan_2009", "PCTP_Jan_2009", "PCCH_Dec08_Jan09", "PCCH_Jan08_Jan09" ]]
cleaned_df.head()

,State,Residents_July_1_2008,Fs_Pop,PCTP_Jan_2008,TP_Dec_2008,PCTP_Dec_2008,TP_Jan_2009,PCTP_Jan_2009,PCCH_Dec08_Jan09,PCCH_Jan08_Jan09
0,Alabama,4661900,564325,12.105043,641449,13.759390,647628,13.891932,1.0,14.8
1,Alaska,686293,56031,8.164297,57411,8.365377,61202,8.917765,6.6,9.2
2,Arizona,6500180,602007,9.261390,742936,11.429468,756960,11.645216,1.9,25.7
3,Arkansas,2855390,379031,13.274229,400571,14.028592,399243,13.982083,-0.3,5.3
4,California,36756666,2160179,5.876972,2502279,6.807688,2545129,6.924265,1.7,17.8


# Database connection

In [62]:

pw = config.Password
connect = ("root:" + pw + "@localhost/snap_db")

engine = create_engine(f'mysql://{connect}')


# Write to Table


In [61]:
cleaned_df.to_sql(name='fs_receipt', con=engine, if_exists='replace', index=False)

pd.read_sql_query('select * from fs_receipt', con=engine).head()

,State,Residents_July_1_2008,Fs_Pop,PCTP_Jan_2008,TP_Dec_2008,PCTP_Dec_2008,TP_Jan_2009,PCTP_Jan_2009,PCCH_Dec08_Jan09,PCCH_Jan08_Jan09
0,Alabama,4661900,564325,12.105043,641449,13.759390,647628,13.891932,1.0,14.8
1,Alaska,686293,56031,8.164297,57411,8.365377,61202,8.917765,6.6,9.2
2,Arizona,6500180,602007,9.261390,742936,11.429468,756960,11.645216,1.9,25.7
3,Arkansas,2855390,379031,13.274229,400571,14.028592,399243,13.982083,-0.3,5.3
4,California,36756666,2160179,5.876972,2502279,6.807688,2545129,6.924265,1.7,17.8
